#End to end multiclass dog breed detection

this notebook builds a multiclass classification model using tensorflow and tensorflow 2.0

##1. Problem

identifying the breed of a dog using a provided image

##2. Data

the data we are using is from a kaggle competition (https://www.kaggle.com/competitions/dog-breed-identification)

##3. Evaluation

the evaluation (or the submission file in this case) is a file containing the probaboilty of being any of the listed dog breeds for every given image

##4. Features

some info about the data:

* were dealing with dog images (ustructured data) so we'll be using deep learning
* there are 120 dog breeds so the model is a multiclass clasifier 
* there are around 10,000 + images in each train and test folder


In [1]:
# unzip the zip folder containg the needed data
# !unzip "/content/drive/MyDrive/Colab Notebooks/dog breed detection project/dog-breed-identification.zip" -d "/content/drive/MyDrive/Colab Notebooks/dog breed detection project/"

## Geting our workspace ready

whats we'll be doing:

* import tensorflow 2.x
* import tensordlow hub
* make sure we are using a gpu
* import other neccasary tools

In [2]:
# importing some tools
import tensorflow as tf
import tensorflow_hub as hub

# check for GPU availability 
print("GPU AVILABLE!!!!!!" if tf.config.list_physical_devices("GPU") else "not available")

GPU AVILABLE!!!!!!
